# simple lgb

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amexdataagg/test_agg_stats.csv
/kaggle/input/amexdataagg/agg_test_all.parquet
/kaggle/input/amexdataagg/agg_train_all.parquet
/kaggle/input/amexdataagg/train_agg_stats.csv
/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv


In [2]:
import numpy as np
import pandas as pd
import gc
import copy
import os
import sys

from pathlib import Path
from datetime import datetime, date, time, timedelta
from dateutil import relativedelta

import pyarrow.parquet as pq
import pyarrow as pa

import dask.dataframe as dd

In [3]:
import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_rows = 100
pd.options.display.max_columns = 100


import pytorch_lightning as pl
random_seed=1234
pl.seed_everything(random_seed)

1234

## load data

In [4]:
%%time
df = pd.read_parquet('/kaggle/input/amexdataagg/agg_train_all.parquet')

CPU times: user 5.07 s, sys: 1.88 s, total: 6.95 s
Wall time: 5.52 s


In [5]:
df.rename(columns={'0':'cnt'}, inplace=True)

In [6]:
%%time
train_labels = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')
print(train_labels.shape)
display(train_labels.head(2))

(458913, 2)


,customer_ID,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0


CPU times: user 529 ms, sys: 181 ms, total: 710 ms
Wall time: 909 ms


### load stats data

In [7]:
agg_stats = pd.read_csv('/kaggle/input/amexdataagg/train_agg_stats.csv', sep='|', index_col=0)

In [8]:
agg_stats.isna().sum()

feat            0
min             0
max             0
mean            0
std             0
median          0
skew            0
kurtosis        0
quantile0.05    0
quantile0.15    0
quantile0.25    0
quantile0.75    0
quantile0.85    0
quantile0.95    0
dtype: int64

In [9]:
agg_stats.head(2)

,feat,min,max,mean,std,median,skew,kurtosis,quantile0.05,quantile0.15,quantile0.25,quantile0.75,quantile0.85,quantile0.95
0,D_144,0.000004,1.342895,0.051679,0.173223,0.005127,4.289567,18.931978,0.003598,0.004168,0.004498,0.005852,0.006462,0.415345
1,S_26,0.000002,84.508010,0.063837,0.411714,0.005889,75.542442,10092.106445,0.003786,0.004434,0.004844,0.032713,0.078652,0.253601


In [10]:
%%time

id_feats = ['customer_ID']
date_col =  'S_2'

all_cols = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_32', 'B_33', 'B_36', 'B_37', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_114=0.0', 'D_114=1.0', 'D_115', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_118', 'D_119', 'D_120=0.0', 'D_120=1.0', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_65', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_2=max', 'S_2=min', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'customer_ID', 'days']

cat_feats = ['B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_31=0', 'B_31=1', 'B_38=1.0', 'B_38=2.0', 'B_38=3.0', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'D_114=0.0', 'D_114=1.0', 'D_116=0.0', 'D_116=1.0', 'D_117=-1.0', 'D_117=1.0', 'D_117=2.0', 'D_117=3.0', 'D_117=4.0', 'D_117=5.0', 'D_117=6.0', 'D_120=0.0', 'D_120=1.0', 'D_126=-1.0', 'D_126=0.0', 'D_126=1.0', 'D_63=CL', 'D_63=CO', 'D_63=CR', 'D_63=XL', 'D_63=XM', 'D_63=XZ', 'D_64=-1', 'D_64=O', 'D_64=R', 'D_64=U', 'D_66=0.0', 'D_66=1.0', 'D_68=0.0', 'D_68=1.0', 'D_68=2.0', 'D_68=3.0', 'D_68=4.0', 'D_68=5.0', 'D_68=6.0']
s2_feats = ['S_2=max', 'S_2=min']
float_feats = ['cnt', 'B_1', 'B_10', 'B_11', 'B_12', 'B_13', 'B_14', 'B_15', 'B_16', 'B_17', 'B_18', 'B_19', 'B_2', 'B_20', 'B_21', 'B_22', 'B_23', 'B_24', 'B_25', 'B_26', 'B_27', 'B_28', 'B_29', 'B_3', 'B_32', 'B_33', 'B_36', 'B_37', 'B_39', 'B_4', 'B_40', 'B_41', 'B_42', 'B_5', 'B_6', 'B_7', 'B_8', 'B_9', 'D_102', 'D_103', 'D_104', 'D_105', 'D_106', 'D_107', 'D_108', 'D_109', 'D_110', 'D_111', 'D_112', 'D_113', 'D_115', 'D_118', 'D_119', 'D_121', 'D_122', 'D_123', 'D_124', 'D_125', 'D_127', 'D_128', 'D_129', 'D_130', 'D_131', 'D_132', 'D_133', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_139', 'D_140', 'D_141', 'D_142', 'D_143', 'D_144', 'D_145', 'D_39', 'D_41', 'D_42', 'D_43', 'D_44', 'D_45', 'D_46', 'D_47', 'D_48', 'D_49', 'D_50', 'D_51', 'D_52', 'D_53', 'D_54', 'D_55', 'D_56', 'D_58', 'D_59', 'D_60', 'D_61', 'D_62', 'D_65', 'D_69', 'D_70', 'D_71', 'D_72', 'D_73', 'D_74', 'D_75', 'D_76', 'D_77', 'D_78', 'D_79', 'D_80', 'D_81', 'D_82', 'D_83', 'D_84', 'D_86', 'D_87', 'D_88', 'D_89', 'D_91', 'D_92', 'D_93', 'D_94', 'D_96', 'P_2', 'P_3', 'P_4', 'R_1', 'R_10', 'R_11', 'R_12', 'R_13', 'R_14', 'R_15', 'R_16', 'R_17', 'R_18', 'R_19', 'R_2', 'R_20', 'R_21', 'R_22', 'R_23', 'R_24', 'R_25', 'R_26', 'R_27', 'R_28', 'R_3', 'R_4', 'R_5', 'R_6', 'R_7', 'R_8', 'R_9', 'S_11', 'S_12', 'S_13', 'S_15', 'S_16', 'S_17', 'S_18', 'S_19', 'S_20', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26', 'S_27', 'S_3', 'S_5', 'S_6', 'S_7', 'S_8', 'S_9', 'days']

print(len(all_cols))
df.info()


228
<class 'pandas.core.frame.DataFrame'>
Int64Index: 458913 entries, 0 to 458912
Columns: 228 entries, customer_ID to D_129
dtypes: float32(176), float64(47), int64(2), object(3)
memory usage: 493.7+ MB
CPU times: user 17.5 ms, sys: 4.14 ms, total: 21.7 ms
Wall time: 23.5 ms


In [11]:
print(len(cat_feats), len(float_feats))

47 178


## process data

- drop columns with 20% data missing
- for float features: fill missing values with mean
- for categorical features:
 - fill values per feature
- log transforma a few columns

In [12]:
for c in cat_feats:
    df[c] = df[c].fillna(0)
    df[c] = df[c].astype('category')
    

In [13]:
n_samples = train_labels.shape[0]

In [14]:
na_cnts = df.isna().sum().to_frame()
na_cnts.columns = ['na_cnt']
na_cnts['na_pct'] = 100*na_cnts['na_cnt']/n_samples

In [15]:
na_cnts.sort_values(by=['na_cnt'], ascending=False)

,na_cnt,na_pct
D_87,458084,99.819356
D_88,456494,99.472885
D_111,454826,99.109417
D_110,454826,99.109417
B_39,454807,99.105277
...,...,...
D_68=0.0,0,0.000000
D_120=0.0,0,0.000000
D_120=1.0,0,0.000000
B_23,0,0.000000


In [16]:
na_cnts[na_cnts['na_pct']>=20]

,na_cnt,na_pct
D_73,450905,98.255007
D_135,424357,92.470032
D_106,397376,86.590705
D_108,437983,95.439223
B_39,454807,99.105277
D_53,314857,68.609301
D_56,214314,46.700355
D_137,424357,92.470032
D_142,373333,81.351585
D_43,103075,22.460684


In [17]:
missing20 = na_cnts[na_cnts['na_pct']>=20].index.tolist()
print(len(missing20))
missing20.sort()
print(missing20)

30
['B_17', 'B_29', 'B_39', 'B_42', 'D_105', 'D_106', 'D_108', 'D_110', 'D_111', 'D_132', 'D_134', 'D_135', 'D_136', 'D_137', 'D_138', 'D_142', 'D_42', 'D_43', 'D_49', 'D_50', 'D_53', 'D_56', 'D_73', 'D_76', 'D_82', 'D_87', 'D_88', 'R_26', 'R_9', 'S_9']


In [18]:
df.drop(columns = missing20, inplace=True)

In [19]:
print('before: ', len(all_cols))
all_cols = list(set(all_cols) - set(missing20))
print('after: ', len(all_cols))

print('before: ', len(float_feats))
float_feats = list(set(float_feats) - set(missing20))
print('after: ', len(float_feats))


before:  228
after:  198
before:  178
after:  148


In [20]:

#fill na
for _, row in agg_stats.iterrows():
    if row['feat'] in float_feats:
        df[row['feat']] = df[row['feat']].fillna(row['mean'])
   
    

In [21]:
df.isna().sum().sort_values(ascending=False)

customer_ID    0
D_86           0
B_20           0
S_13           0
D_126=-1.0     0
              ..
B_30=1.0       0
B_30=2.0       0
B_27           0
D_66=0.0       0
D_129          0
Length: 198, dtype: int64

### combine train data with train labels



In [22]:
df.shape, train_labels.shape, id_feats

((458913, 198), (458913, 2), ['customer_ID'])

In [23]:
df = df.merge(train_labels, on=id_feats, how='inner')
df.shape

(458913, 199)

In [24]:
del train_labels
gc.collect()

21

In [25]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

## train lightgbm models

In [26]:
import lightgbm as lgb
import json
import pickle
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from lightgbm import LGBMClassifier
from sklearn.preprocessing import label_binarize
import copy


def train_trees(X_train, y_train, num_round=100, params={}, 
                verbosity=-1, 
                save_model = False, model_file = None):
    
    dtrain = lgb.Dataset(X_train, y_train)
    params['verbosity']=verbosity
    
    tree_model = lgb.train(params,
                dtrain,
                num_boost_round=num_round
                          )
                           
    if save_model:
        if model_file is None:
            model_file = 'lgb.pkl'
        with open(model_file, 'wb') as fout:
            pickle.dump(tree_model, fout)
   
    del dtrain, tree_model
    gc.collect()


In [27]:
binary_params = [
#     {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.007, 'max_bin': 1650, 'max_depth': 14, 'metric': 'auc', 'min_child_samples': 4200, 'min_data_in_bin': 6650, 'n_estimators': 1700, 'n_jobs': 4, 'num_leaves': 111, 'objective': 'binary', 'random_state': 8696, 'reg_alpha': 15, 'reg_lambda': 0.01, 'subsample': 0.1, 'subsample_freq': 11},
#     {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.007, 'max_bin': 1650, 'max_depth': 14, 'metric': 'auc', 'min_child_samples': 4200, 'min_data_in_bin': 6650, 'n_estimators': 1700, 'n_jobs': 4, 'num_leaves': 111, 'objective': 'binary', 'random_state': 1, 'reg_alpha': 15, 'reg_lambda': 0.01, 'subsample': 0.1, 'subsample_freq': 11},
#     {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.007, 'max_bin': 1650, 'max_depth': 14, 'metric': 'auc', 'min_child_samples': 4200, 'min_data_in_bin': 6650, 'n_estimators': 1700, 'n_jobs': 4, 'num_leaves': 111, 'objective': 'binary', 'random_state': 33, 'reg_alpha': 15, 'reg_lambda': 0.01, 'subsample': 0.1, 'subsample_freq': 11},
#     {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.007, 'max_bin': 1650, 'max_depth': 14, 'metric': 'auc', 'min_child_samples': 4200, 'min_data_in_bin': 6650, 'n_estimators': 1700, 'n_jobs': 4, 'num_leaves': 111, 'objective': 'binary', 'random_state': 666, 'reg_alpha': 15, 'reg_lambda': 0.01, 'subsample': 0.1, 'subsample_freq': 11},
#     {'boosting_type': 'gbdt', 'colsample_bytree': 0.4, 'learning_rate': 0.007, 'max_bin': 1650, 'max_depth': 14, 'metric': 'auc', 'min_child_samples': 4200, 'min_data_in_bin': 6650, 'n_estimators': 1700, 'n_jobs': 4, 'num_leaves': 111, 'objective': 'binary', 'random_state': 12345, 'reg_alpha': 15, 'reg_lambda': 0.01, 'subsample': 0.1, 'subsample_freq': 11},
    {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.0004, 'max_bin': 750, 'max_depth': 15, 'metric': 'auc', 'min_child_samples': 1500, 'min_data_in_bin': 4450, 'n_estimators': 500, 'n_jobs': 4, 'num_leaves': 81, 'objective': 'binary', 'random_state': 8696, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.1, 'subsample_freq': 45},
    {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.0004, 'max_bin': 750, 'max_depth': 15, 'metric': 'auc', 'min_child_samples': 1500, 'min_data_in_bin': 4450, 'n_estimators': 500, 'n_jobs': 4, 'num_leaves': 81, 'objective': 'binary', 'random_state': 1, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.1, 'subsample_freq': 45},
    {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.0004, 'max_bin': 750, 'max_depth': 15, 'metric': 'auc', 'min_child_samples': 1500, 'min_data_in_bin': 4450, 'n_estimators': 500, 'n_jobs': 4, 'num_leaves': 81, 'objective': 'binary', 'random_state': 21, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.1, 'subsample_freq': 45},
    {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.0004, 'max_bin': 750, 'max_depth': 15, 'metric': 'auc', 'min_child_samples': 1500, 'min_data_in_bin': 4450, 'n_estimators': 500, 'n_jobs': 4, 'num_leaves': 81, 'objective': 'binary', 'random_state': 345, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.1, 'subsample_freq': 45},
    {'boosting_type': 'gbdt', 'colsample_bytree': 0.65, 'learning_rate': 0.0004, 'max_bin': 750, 'max_depth': 15, 'metric': 'auc', 'min_child_samples': 1500, 'min_data_in_bin': 4450, 'n_estimators': 500, 'n_jobs': 4, 'num_leaves': 81, 'objective': 'binary', 'random_state': 90000, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.1, 'subsample_freq': 45},
    
]

model_files = ['lgb1.pkl', 'lgb2.pkl', 'lgb3.pkl', 'lgb4.pkl', 'lgb5.pkl']

In [28]:
%%time
for params, model_file in zip(binary_params, model_files):
    num_boost_round = params['n_estimators']
    params_ = copy.deepcopy(params)
    del params_['n_estimators']


    #-----------------lgb train start-------------------------------------

    train_trees(df[float_feats + cat_feats].values, df['target'].values, 
                num_round=num_boost_round, params=params_, 
                verbosity=-1, 
                save_model = True, model_file = model_file)

CPU times: user 9min 41s, sys: 4.32 s, total: 9min 46s
Wall time: 2min 39s


## Inference

In [29]:
def make_preds(X, model_files):
    
    pred_list=[]

    for model_file in model_files:
        with open(model_file, 'rb') as f_in:
            tree_model = pickle.load(f_in)
            y_preds = tree_model.predict(X, num_iteration=tree_model.best_iteration)
            
        pred_list.append(y_preds)
    
    
    return np.array(pred_list)

In [30]:
def process_features(test_file):
    data = pd.read_parquet(test_file)
    data.rename(columns={'0':'cnt'}, inplace=True)
    #drop columns with 20% missing values
    data.drop(columns=missing20, inplace=True)
    
    cur_cols = data.columns.tolist()
#     print(cur_cols)
    
    #fill na
    for _, row in agg_stats.iterrows():
        if row['feat'] in float_feats:
            if row['feat'] in cur_cols:
                data[row['feat']] = data[row['feat']].fillna(row['mean'])
        

    for c in cat_feats:
        if c in cur_cols:
            data[c] = data[c].fillna(0)   
        else:
            data[c] = 0
        data[c] = data[c].astype('category')  
        
    return data[id_feats + float_feats + cat_feats]

In [31]:
submission_file = '/kaggle/input/amex-default-prediction/sample_submission.csv'
test_file = '/kaggle/input/amexdataagg/agg_test_all.parquet'

In [32]:
%%time
data = process_features(test_file)
all_preds = make_preds(data[float_feats + cat_feats], model_files)

CPU times: user 4min 7s, sys: 11.1 s, total: 4min 18s
Wall time: 1min 21s


In [33]:
%%time
test_preds = pd.DataFrame(data={'customer_ID': data['customer_ID'].values, 'prediction':all_preds.mean(axis=0)})


submission = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
submission = submission[['customer_ID']].merge(test_preds[['customer_ID', 'prediction']], 
                                               on=['customer_ID'], how='left')

submission.isna().sum()

CPU times: user 2.19 s, sys: 105 ms, total: 2.29 s
Wall time: 2.92 s


customer_ID    0
prediction     0
dtype: int64

In [34]:
del data, all_preds
gc.collect()

46

In [35]:
submission.fillna(value=0, inplace=True)
submission.to_csv('submission.csv', index=False)